# Prompts
Examples working to better understand prompts in langchain. Reference: [Prompts](https://python.langchain.com/docs/modules/model_io/prompts/)

In [5]:
# VSCode will prompt for API Key
openai_api_key = input("Enter OpenAI API key: ")

## Prompt Template
Building off of the examples given in the quick start. This example has multiple inputs and does not use an output parser.

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain.prompts import (
    PromptTemplate
)
from langchain.chains import LLMChain

# setup the chat model
chat_model = ChatOpenAI(openai_api_key=openai_api_key)

### Ghostwriter

Ghostwriter needs multiple prompts that can be chained together:
- Write a blog post, in the first person, based on a description by the user. Include markdown image placeholders with descriptions of what the image should be.
- Write a concluding paragraph with the heading "Conclusion / TL;DR;" that has two sentences that summarize the article and a bullet point list with the main takeaways from the article.
- Write YAML front matter for the article

In [18]:
ghostwriter_template = """You are a ghostwriter that helps technical bloggers write full-length markdown blog posts.
A user will give you an {article_summary} and {style} in which to write the article. Write a full length blog post based on the summary provided and include image placeholders with description of a sample image that should go in that placeholder.
The first line should be the title of the article.
The opening sentence should be: Hello friendos, it's DW.
The closing paragraph should have the heading \"Conclusion \\ TL;DR;\" and have a one or two sentence summary of the post, and include bullet points with the key takeaways.
The last line should be: Thanks for playing. 
"""
ghostwriter_prompt = SystemMessagePromptTemplate.from_template(ghostwriter_template)

# setup the chat model
chat_model = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-4")

# setup ghostwriter prompt
ghostwriter_chat_prompt = ChatPromptTemplate.from_messages([ghostwriter_prompt])
ghostwriter_chain = LLMChain(
    llm=chat_model,
    prompt=ghostwriter_chat_prompt,
    output_key="article"
)

'''
output = chain.run(
    article_summary="I started a new podcast in July 2023 called \"Remember the Human\" which is an AI podcast about change it brings to the day-to-day lives of people. I started it because I wanted to hear stories about the impact AI has, rather than the technology itself, as media tends to focus on the change AI could bring, rather than report on the changes it has already brought. The podcast is available on YouTube, Spotify, and Apple Podcasts and you can join the conversation on the LinkedIn page or the official subreddit /r/rememberthehuman.",
    style="Casual, as though you are having a conversation with a friend over coffee."
)
print(output)
'''

'\noutput = chain.run(\n    article_summary="I started a new podcast in July 2023 called "Remember the Human" which is an AI podcast about change it brings to the day-to-day lives of people. I started it because I wanted to hear stories about the impact AI has, rather than the technology itself, as media tends to focus on the change AI could bring, rather than report on the changes it has already brought. The podcast is available on YouTube, Spotify, and Apple Podcasts and you can join the conversation on the LinkedIn page or the official subreddit /r/rememberthehuman.",\n    style="Casual, as though you are having a conversation with a friend over coffee."\n)\nprint(output)\n'

In [21]:
# setup front matter prompt
frontmatter_template = """You are a copywriter assistant that writes YAML front matter for markdown articles.
You will receive an {article} from a user, and you will create the YAML front matter for the article with the following properties: title, description, tags (list)
First line should be: ---
Last line should be: ---
"""
frontmatter_prompt = SystemMessagePromptTemplate.from_template(frontmatter_template)
frontmatter_chat_prompt = ChatPromptTemplate.from_messages([frontmatter_prompt])
frontmatter_chain = LLMChain(
    llm=chat_model,
    prompt=frontmatter_chat_prompt,
    output_key="frontmatter"
)

In [27]:
from langchain.chains import (
    SequentialChain
)
overall_chain = SequentialChain(
    chains=[ghostwriter_chain, frontmatter_chain],
    input_variables=["article_summary", "style"],
    output_variables=["article", "frontmatter"],
    verbose=False
)
output = overall_chain({
    "article_summary":"I started a new podcast in July 2023 called \"Remember the Human\" which is an AI podcast about change it brings to the day-to-day lives of people. I started it because I wanted to hear stories about the impact AI has, rather than the technology itself, as media tends to focus on the change AI could bring, rather than report on the changes it has already brought. The podcast is available on YouTube, Spotify, and Apple Podcasts and you can join the conversation on the LinkedIn page or the official subreddit /r/rememberthehuman.",
    "style":"Casual, as though you are having a conversation with a friend over coffee."
})

print(output['frontmatter'])
print(output['article'])

---
title: "My Journey to Starting an AI Podcast: 'Remember the Human'"
description: "In this article, the author describes the journey of starting an AI-centric podcast called 'Remember the Human.' The podcast discusses the current impact of AI on our lives, showcasing stories from experts, innovators, and everyday people."
tags: ["AI", "Podcast", "Technology", "Impact", "Innovation"]
---
# My Journey to Starting an AI Podcast: "Remember the Human"

Hello friendos, it's DW. Sip your coffee, and let's talk about something cool I started: a podcast called "Remember the Human." It's all about the impact of Artificial Intelligence on our everyday lives. 

![Placeholder: A logo of the podcast "Remember the Human" featuring a stylized human and AI motif]

I kicked off this podcast in July 2023 because I wanted to hear and share stories about the impact AI has on us. Not the sci-fi, futuristic, robots-taking-over-the-world kind of stories, but real, tangible ones that show how AI is already 